divide and conquer

- transform into a problem we already know how to solve, or that we understand
- to detect duplicates
- first, sort
- then, check adjacent elements

## Recursion

In [2]:
def factorial1(n):
    result = 1
    for i in range(n):
        result *= (i+1)
    return result

In [3]:
def factorial2(n):
    if n == 1:
        return 1
    else:
        return n * factorial2(n-1)

In [4]:
factorial1(0)

1

In [6]:
factorial2(1)

1

In [7]:
def size1(s):
    return 1 + sum(size1(child) for child in s.hyponyms())

In [12]:
# longer and harder to understand
# iterative
# procedural
def size2(s):
    layer = [s]
    total = 0
    while layer:
        total += len(layer)
        layer = [h for c in layer for h in c.hyponyms()]
    return total

In [9]:
from nltk.corpus import wordnet as wn
dog = wn.synset('dog.n.01')

In [10]:
size1(dog)

190

In [13]:
size2(dog)

190

In [14]:
# retrieval
# letter trie

In [15]:
def insert(trie, key, value):
    if key:
        first, rest = key[0], key[1:]
        if first not in trie:
            trie[first] = {}
        insert(trie[first], rest, value)
    else:
        trie['value'] = value

In [16]:
trie = {}
insert(trie, 'chat', 'cat')
insert(trie, 'chien', 'dog')
insert(trie, 'chair', 'flesh')
insert(trie, 'chic', 'stylish')

In [17]:
trie = dict(trie) # for nicer printing

In [18]:
trie['c']['h']['a']['t']['value']

'cat'

In [20]:
import pprint
pprint.pprint(trie, width=40)

{'c': {'h': {'a': {'i': {'r': {'value': 'flesh'}},
                   't': {'value': 'cat'}},
             'i': {'c': {'value': 'stylish'},
                   'e': {'n': {'value': 'dog'}}}}}}


- simplicity
- expensive, each loop -> state info into the stack

## Space-Time Tradeoffs

In [23]:
import nltk
from nltk import re

def raw(file):
    contents = open(file).read()
    contents = re.sub(r'<.*?>', ' ', contents)
    contents = re.sub('\s+', ' ', contents)
    return contents

def snippet(doc, term):
    text = ' '*30 + raw(doc) + ' '*30
    pos = text.index(term)
    return text[pos-30:pos+30]

print("Building Index...")
files = nltk.corpus.movie_reviews.abspaths()
idx = nltk.Index((w, f) for f in files for w in raw(f).split())

query = ''
while query != "quit":
    query = input("query> ")     # use raw_input() in Python 2
    if query in idx:
        for doc in idx[query]:
            print(snippet(doc, query))
    else:
        print("Not found")

Building Index...
query> titanic
e flood tale could have had . titanic proved that there can 
y oscar-winner james horner ( titanic ) , one might have exp
ip -- no , it wouldn't be the titanic , that would imply gla
 check out whatever's left of titanic ) , the virtually non-
 check out whatever's left of titanic ) , the virtually non-
els like tremors grafted onto titanic ( everyone else is cit
els like tremors grafted onto titanic ( everyone else is cit
els like tremors grafted onto titanic ( everyone else is cit
though it feels longer than " titanic " , and is rated r for
io for the thirteenth time in titanic to catch a glimpse of 
-- if these people had seen " titanic , " then maybe they'd 
-- if these people had seen " titanic , " then maybe they'd 
 when it's half the length of titanic , but seems much longe
hat kate winslet floats on in titanic . it's an exceptionlly
y bad . it stars billy zane ( titanic ) as a dangerous lunat
 suffer from its proximity to titanic . deep rising'

In [24]:
def preprocess(tagged_corpus):
    words = set()
    tags = set()
    for sent in tagged_corpus:
        for word, tag in sent:
            words.add(word)
            tags.add(tag)
    wm = dict((w, i) for (i, w) in enumerate(words))
    tm = dict((t, i) for (i, t) in enumerate(tags))
    return [[(wm[w], tm[t]) for (w, t) in sent] for sent in tagged_corpus]

In [ ]:
from timeit import Timer
vocab_size = 1000
setup_list = "import random; vocab = range(%d)" % vocab_size
setup_set = "import random; vocab = set(range(%d))" % vocab_size
statement = "random.randint(0, %d) in vocab" % (vocab_size * 2)

In [ ]:
print(Timer(statement, setup_list).timeit(1000))

In [ ]:
print(Timer(statement, setup_set).timeit(1000))

## Dynamic Programming

In [2]:
def virahanka1(n):
    if n == 0:
        return [""]
    elif n == 1:
        return ["S"]
    else:
        s = ["S" + prosody for prosody in virahanka1(n-1)]
        l = ["L" + prosody for prosody in virahanka1(n-2)]
        return s + l

def virahanka2(n):
    lookup = [[""], ["S"]]
    for i in range(n-1):
        s = ["S" + prosody for prosody in lookup[i+1]]
        l = ["L" + prosody for prosody in lookup[i]]
        lookup.append(s + l)
    return lookup[n]

def virahanka3(n, lookup={0:[""], 1:["S"]}):
    if n not in lookup:
        s = ["S" + prosody for prosody in virahanka3(n-1)]
        l = ["L" + prosody for prosody in virahanka3(n-2)]
        lookup[n] = s + l
    return lookup[n]

from nltk import memoize
@memoize
def virahanka4(n):
    if n == 0:
        return [""]
    elif n == 1:
        return ["S"]
    else:
        s = ["S" + prosody for prosody in virahanka4(n-1)]
        l = ["L" + prosody for prosody in virahanka4(n-2)]
        return s + l

In [3]:
virahanka1(4)

['SSSS', 'SSL', 'SLS', 'LSS', 'LL']

In [4]:
virahanka2(4)

['SSSS', 'SSL', 'SLS', 'LSS', 'LL']

In [5]:
virahanka3(4)

['SSSS', 'SSL', 'SLS', 'LSS', 'LL']

In [6]:
virahanka4(4)

['SSSS', 'SSL', 'SLS', 'LSS', 'LL']